In [1]:
# import needed packages
from ptk import *

In [2]:
# create a stochastic variable
stochast = Stochast()

# modify the stochastic variables
# supported distributions (so far): deterministic, normal, log_normal, uniform, exponential, gumbel, weibull, conditional_weibull, frechet, generalized_extreme_value, rayleigh, rayleigh_n, discrete, histogram, cdf_curve
# available soon: triangular, trapezoidal, beta, gamma, pareto, generalized_pareto, bernoulli, poisson, student_t
stochast.distribution = 'uniform'

# set stochastic properties: location, scale, shape, shape_b, shift, shift_b, minimum, maximum, observations
stochast.minimum = -1
stochast.maximum = 1

# the mean and deviation (= std. dev.) are derived properties, but can be set too
print('uniform stochast, mean = ' + str(stochast.mean) + ', std dev = ' + str(stochast.deviation))

# get the x value corresponding to a non-exceeding probability of 0.75
u = StandardNormal.get_u_from_p(0.75)
print('uniform, x(0.75) = ' + str(stochast.get_x_from_u(u)))

# fit the stochast properties
data = [2.3, 2.6, 2.7, 2.8, 3.3, 3.4]
stochast.fit(data)

print('fitted, mean = ' + str(stochast.mean) + ', std dev = ' + str(stochast.deviation))

# change the distribution, mean and deviation are retained, other properties are modified
stochast.distribution = 'log_normal'

print('lognormal, mean = ' + str(stochast.mean) + ', std dev = ' + str(stochast.deviation))
print('lognormal, location = ' + str(stochast.location) + ', scale = ' + str(stochast.scale))
print('lognormal, x(0.75) = ' + str(stochast.get_x_from_u(u)))

# invert the distribution
stochast.inverted = True

print('lognormal inverted, mean = ' + str(stochast.mean) + ', std dev = ' + str(stochast.deviation))
print('lognormal inverted, location = ' + str(stochast.location) + ', scale = ' + str(stochast.scale))
print('lognormal inverted, x(0.25) = ' + str(stochast.get_x_from_u(-u)))

# truncate the distribution
stochast.truncated = True
stochast.minimum = -999999 # no truncation on the low side
stochast.maximum = -3
print('lognormal inverted truncated, x(0.25) = ' + str(stochast.get_x_from_u(-u)))



uniform stochast, mean = 0.0, std dev = 0.5773502691896257
uniform, x(0.75) = 0.5
fitted, mean = 2.8499999999999996, std dev = 0.42339019740572564
lognormal, mean = 2.8499999999999996, std dev = 0.4233901974057255
lognormal, location = 1.0364042629164547, scale = 0.14774797030148581
lognormal, x(0.75) = 3.11446925625268
lognormal inverted, mean = -2.8499999999999996, std dev = 0.4233901974057255
lognormal inverted, location = 1.0364042629164547, scale = 0.14774797030148581
lognormal inverted, x(0.25) = -3.11446925625268
lognormal inverted truncated, x(0.25) = -3.4552401341510737


In [3]:
# define model
def linear(a, b):
    return 1.9 - (a+b)

In [4]:
# perform a reliability analysis

# always create a reliablity project
project = ReliabilityProject()

# the fisrst thing to do is specify the model
project.model = linear

# when the model is specified, stochastic variables are generated
[print(i) for i in project.variables]

stochast1 = project.variables['a']
stochast1.distribution = 'uniform'
stochast1.minimum = -1
stochast1.maximum = 1

stochast2 = project.variables['b']
stochast2.distribution = 'uniform'
stochast2.minimum = -1
stochast2.maximum = 1

# specify the reliability algorithm (form, numerical_integration, crude_monte_carlo, importance_sampling, adaptive_importance_sampling, subset_simulation)
project.settings.reliability_method = 'form'

# set the settings for the calculation (variation_coefficient, relaxation_factor/loops, minimum/maximum_samples/iterations/directions/variance_loops, fraction_failed)
project.settings.variation_coefficient = 0.02
project.settings.maximum_iterations = 20

# execute the reliability analysis
project.run()

# the result is written in the design point
dp = project.design_point
beta = dp.reliability_index

print('beta = ' + str(beta))

# convert to probability of failure
pf = StandardNormal.get_q_from_u(beta)
print('probability of failure = ' + str(pf))

# check whether converged
if dp.is_converged:
    print('Converged, convergence = ' + str(dp.convergence) + ' < ' + str(project.settings.variation_coefficient))
else:
    print('Not converged, convergence = ' + str(dp.convergence) + ' > ' + str(project.settings.variation_coefficient))


a
b
beta = 2.772351852475011
probability of failure = 0.0027826422680776335
Converged, convergence = 0.004050836299007645 < 0.02


In [5]:
# alpha values are available in the design point
for alpha in dp.alphas:
    print(alpha.variable.name + ': alpha = ' + str(alpha.alpha) + ', x = ' + str(alpha.x))


a: alpha = -0.7071067811865476, x = 0.9500449635563615
b: alpha = -0.7071067811865476, x = 0.9500449635563615


In [6]:
# apply correlation
project.correlation_matrix[('a', 'b')] = 0.8

project.run()

dp = project.design_point
beta = dp.reliability_index

print('beta = ' + str(beta))

for alpha in dp.alphas:
    print(alpha.variable.name + ': alpha = ' + str(alpha.alpha) + ', x = ' + str(alpha.x))


beta = 2.0665776344233597
a: alpha = -0.9456009795757881, x = 0.9493174459367089
b: alpha = -0.3253287374722835, x = 0.950784099184725


In [7]:
# define another model
def linear_2(b, c):
    return 1.85 - (1.5 * b + 0.5 * c)

project.model = linear_2

# b is the same variable as used before
print(str(project.variables['b']) + ': ' + project.variables['b'].distribution)

project.variables['c'].distribution = 'normal'
project.variables['c'].mean = 0.1
project.variables['c'].deviation = 0.8

project.run()

dp2 = project.design_point

print('beta = ' + str(dp2.reliability_index))


b: uniform
beta = 1.9500014680701803


In [8]:
# combine the design points

# create a new type of project 
combine_project = CombineProject()

combine_project.design_points.append(dp)
combine_project.design_points.append(dp2)

# specify the combine algorithm (hohenbichler, importance_sampling, directional sampling)
combine_project.settings.combiner_method = 'hohenbichler' 

# specify the combination type (series = or, parallel = and)
combine_project.settings.combine_type = 'series'

# execute the combination algorithm
combine_project.run()

dp_correlated = combine_project.design_point
print('beta combined = ' + str(dp_correlated.reliability_index))


beta combined = 1.7119306414567843


In [9]:
# combine the design points with uncorrelated b values

# set the self correlation of stochast b, default value is 1
combine_project.correlation_matrix['b'] = 0

combine_project.run()

dp_uncorrelated = combine_project.design_point
print('beta combined uncorrelated = ' + str(dp_uncorrelated.reliability_index))


beta combined uncorrelated = 1.7009212971591265
